In [1]:
!pip install -r ../requirements.txt

  Using cached cohere-4.32-py3-none-any.whl (47 kB)
  Using cached tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl (924 kB)
  Using cached openai-0.28.1-py3-none-any.whl (76 kB)
  Using cached text_generation-0.6.1-py3-none-any.whl (10 kB)
  Using cached gradio-3.50.2-py3-none-any.whl (20.3 MB)
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
  Using cached datasets-2.14.6-py3-none-any.whl (493 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:00a 0:00:01
  Using cached faiss_cpu-1.7.4-cp311-cp311-macosx_11_0_arm64.whl (2.7 MB)
  Using cached bitsandbytes-0.41.1-py3-none-any.whl (92.6 MB)
  Using cached pandas-2.1.2-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl (9.4 MB)
  Using cached accelerate-0.24.0-py3-none-any.whl (260 kB)
  Using cached tr

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import openai
import pandas as pd
import gradio as gr
import requests, json
from text_generation import Client
from bs4 import BeautifulSoup
import PyPDF2
import io
import tiktoken
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import random
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from datasets import Dataset, load_dataset
import transformers
import torch
from accelerate import Accelerator



In [ ]:
### Run Only when using colab for formatting table display
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
### Run Only when using colab for formatting table display
%load_ext google.colab.data_table

Mounted at /content/drive/


In [ ]:
#from google.colab import drive
drive.mount('/content/drive/')

In [4]:
data_folder_path='../Data/'
training_data_file='fine_tuning_data.csv'

In [5]:
data_for_finetuning=pd.read_csv(data_folder_path+training_data_file)

In [6]:
import json
import re
import torch
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from datasets import Dataset, load_dataset
from accelerate import Accelerator


device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

MODEL_NAME="meta-llama/Llama-2-7b-hf"

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [7]:
print(device)

cpu


In [8]:
def training_prompt(input,output):
  return f"""### Input: {input}

### Output: {output}"""

In [9]:
print(training_prompt(data_for_finetuning.iloc[0]['prompt'],data_for_finetuning.iloc[0]['completion']))

### Input: share my knowledge on The Influence Of Typography On Social Media Posts

### Output:  Typography is not everything that will improve your social media, it's the only thing


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [11]:
from configparser import ConfigParser

config_object = ConfigParser()
config_object.read("../config.ini")
hf_token=config_object["HuggingFace"]["hf_token"]

In [13]:
def create_model_and_tokenizers():
  bnb_config=BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.float16
  )
  model=AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      use_safetensors=False,
      quantization_config=bnb_config,
      trust_remote_code=True,
      device_map="auto",
      token=hf_token
  )
  tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
  tokenizer.pad_token=tokenizer.eos_token
  tokenizer.padding_side="right"

  return model, tokenizer

In [14]:
model, tokenizer = create_model_and_tokenizers()
model.config.use_cache=False

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [ ]:
def tokenize_function(examples):
    text = training_prompt(str(examples['prompt']),str(examples['completion']))

    return text

In [ ]:
data_for_finetuning['text']=data_for_finetuning.apply(tokenize_function, axis=1)

In [ ]:
finetuning_dataset_loaded=Dataset.from_pandas(data_for_finetuning)

In [ ]:
finetuning_dataset_loaded

Dataset({
    features: ['prompt', 'completion', 'text'],
    num_rows: 481
})

In [ ]:
split_dataset = finetuning_dataset_loaded.train_test_split(test_size=0.2, shuffle=True, seed=145)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'text'],
        num_rows: 384
    })
    test: Dataset({
        features: ['prompt', 'completion', 'text'],
        num_rows: 97
    })
})


In [ ]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [ ]:
lora_alpha=32
lora_dropout=.05
lora_r=16

peft_config=LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
Output_dir="experiments"

%load_ext tensorboard
%tensorboard --logdir experiments/runs

<IPython.core.display.Javascript object>

In [ ]:
training_arguments=TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy='steps',
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy='epoch',
    group_by_length=True,
    output_dir=Output_dir,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=145
)

In [ ]:
trainer=SFTTrainer(
    model=model,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments
)

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,2.920300,2.482079
20,1.675000,1.757244
30,1.723300,1.264476
40,1.139900,1.020264


TrainOutput(global_step=48, training_loss=1.8998434953391552, metrics={'train_runtime': 77.9838, 'train_samples_per_second': 9.848, 'train_steps_per_second': 0.616, 'total_flos': 1279270908690432.0, 'train_loss': 1.8998434953391552, 'epoch': 2.0})

In [ ]:
save_dir = 'experiments/final'

trainer.save_model(save_dir)

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linea

In [ ]:
def inference(model,prompt):
  inputs=tokenizer(prompt,
          return_tensors="pt").to(DEVICE)

  input_length=len(inputs['input_ids'][0])
  with torch.inference_mode():
    outputs=model.generate(**inputs,max_new_tokens=256,temperature=0.01)

  return tokenizer.decode(outputs[0][input_length:])

In [ ]:
model, tokenizer = create_model_and_tokenizers()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inference(model,"share information about Benefits Of Mindfulness Meditation")

'\n nobody is perfect, and we all make mistakes. But if we are willing to learn from our mistakes, we can become better people.\nMindfulness meditation is a practice that has been around for centuries. It is a way of focusing your attention on the present moment, without judgment.\nThere are many benefits of mindfulness meditation, including:\n1. Reduced stress and anxiety\n2. Improved focus and concentration\n3. Increased self-awareness\n4. Improved sleep quality\n5. Improved physical health\n6. Improved mental health\n7. Increased compassion and empathy\n8. Increased self-esteem\n9. Increased resilience\n10. Increased happiness\n11. Increased life satisfaction\n12. Increased sense of purpose\n13. Increased sense of meaning\n14. Increased sense of connection\n15. Increased sense of peace\n16. Increased sense of joy\n17. Increased sense of gratitude\n18. Increased sense of hope\n19. Incre'

In [ ]:
model=AutoModelForCausalLM.from_pretrained("experiments/final", local_files_only=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inference(model,"share information about Benefits Of Mindfulness Meditation")